# Objectives

1. Animated Subtree

1. Sampling Bias
    - A. Full
    - B. Russia "Knockout"
    - C. Germany "Knockout"
1. Geocoding Bias
1. Drawing Bias

---
# Setup

## Modules

In [ ]:
import os
import dill
import copy
import math

from Bio import Phylo
import pandas as pd
from shapely.geometry import Polygon
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from matplotlib import lines, patheffects, animation, gridspec

## Variables

In [ ]:
from config import *

# Custom script variables
SCRIPT_NAME = "geo"
PREV_DIR_NAME = "mugration"
PREV_SCRIPT_NAME = "mugration_model"

try:
    WILDCARDS = snakemake.wildcards
    project_dir = os.getcwd()
except NameError:
    WILDCARDS = ["all", "chromosome", "5"]
    project_dir = os.path.dirname(os.path.dirname(os.getcwd()))
    
READS_ORIGIN = WILDCARDS[0]
LOCUS_NAME = WILDCARDS[1]
MISSING_DATA = WILDCARDS[2]

BASEMAP = cimgt.Stamen('terrain-background')
NAME_COL = "Name"

In [ ]:
import sys, importlib
importlib.reload(sys.modules['config'])
from config import *

## File Paths

In [ ]:
config_dir = os.path.join(project_dir, "config")
results_dir = os.path.join(project_dir, "results")

outdir       = os.path.join(results_dir, SCRIPT_NAME, READS_ORIGIN, LOCUS_NAME + "_filter{}".format(MISSING_DATA),)
#tree_dill    = os.path.join(results_dir, PREV_DIR_NAME, READS_ORIGIN, LOCUS_NAME + "_filter{}".format(MISSING_DATA),PREV_SCRIPT_NAME + "_timetree.treetime.obj")
#tree_df_dill = os.path.join(results_dir, PREV_DIR_NAME, READS_ORIGIN, LOCUS_NAME + "_filter{}".format(MISSING_DATA), PREV_SCRIPT_NAME + ".df.obj")
tree_path    = os.path.join(results_dir, PREV_DIR_NAME, READS_ORIGIN, LOCUS_NAME + "_filter{}".format(MISSING_DATA), PREV_SCRIPT_NAME + "_timetree.nwk")
tree_df_path = os.path.join(results_dir, PREV_DIR_NAME, READS_ORIGIN, LOCUS_NAME + "_filter{}".format(MISSING_DATA), PREV_SCRIPT_NAME + ".tsv")
aln_path     = os.path.join(results_dir,"snippy_multi",READS_ORIGIN,"snippy-core_{}.snps.filter{}.aln".format(LOCUS_NAME, MISSING_DATA))

# Auspice
auspice_latlon_path = os.path.join(results_dir, "parse_tree", READS_ORIGIN, LOCUS_NAME + "_filter{}".format(MISSING_DATA), "parse_tree" + "_latlon.tsv")
auspice_colors_path = os.path.join(results_dir, "parse_tree", READS_ORIGIN, LOCUS_NAME + "_filter{}".format(MISSING_DATA), "parse_tree" + "_colors.tsv")
auspice_config_path = os.path.join(config_dir, "auspice_config.json")
auspice_remote_dir_path = os.path.join(project_dir, "auspice/")

#print("tree_dill:\t", tree_dill)
#print("tree_df_dill:\t", tree_df_dill)
print("aln path:\t", aln_path)
print("auspice_latlon_path:", auspice_latlon_path)
print("auspice_colors_path:", auspice_colors_path)
print("auspice_config_path:", auspice_config_path)
print("auspice_remote_dir_path:", auspice_remote_dir_path)
print("outdir:", outdir)

# Create output directory if it doesn't exist
while not os.path.exists(outdir):
    os.makedirs(outdir)   
    
SCRIPT_NAME = "geo_animate"

## Import Tree

In [ ]:
# with open(tree_dill, "rb") as infile:
#     tt = dill.load(infile)
# tt.tree.ladderize(reverse=False)

tree = Phylo.read(tree_path, "newick")
tree.ladderize(reverse=False)

## Import Dataframe

In [ ]:
# with open(tree_df_dill, "rb") as infile:
#     tree_df = dill.load(infile)
# display(tree_df)

tree_df = pd.read_csv(tree_df_path, sep='\t')
tree_df.fillna(NO_DATA_CHAR, inplace=True)
tree_df.set_index(NAME_COL, inplace=True)

# Import Colors

In [ ]:
colors_df = pd.read_csv(auspice_colors_path, sep='\t', header=None)
colors_df.columns = ["Attribute", "State", "Color"]
colors_df.fillna(NO_DATA_CHAR, inplace=True)

# Add sizes
colors_df["Size"] = [NO_DATA_CHAR for i in range(0,len(colors_df))]
i = 0
for attr in list(dict.fromkeys(colors_df["Attribute"])):
    title_attr = attr.replace("_"," ").title().replace(" ","_")
    attr_df =  colors_df[colors_df["Attribute"] == attr]
    for state in attr_df["State"]:
        if attr == "branch_number":
            state = int(state)
        matches = tree_df[tree_df[title_attr] == state]
        colors_df.at[i,"Size"] = len(matches)
        i += 1

display(colors_df)
    
# Convert to dict
colors_dict = {}
sizes_dict = {}
for rec in colors_df.iterrows():
    attr = rec[1]["Attribute"]
    state = rec[1]["State"]
    color = rec[1]["Color"]
    size = rec[1]["Size"]

    if attr not in colors_dict:
        colors_dict[attr] = {}
    if attr not in sizes_dict:
        sizes_dict[attr] = {}        
    colors_dict[attr][state] = color        
    sizes_dict[attr][state] = size

---
# 1. Animate Subtree

In [ ]:
branch   = "1.PRE"
geo_attr = "Province"

## Create Subtree and Dataframe

In [ ]:
clade_df = tree_df[tree_df["Mugration_Branch_Major"] == branch]
clade = tree.common_ancestor(clade_df.index)
parent_node = copy.deepcopy(get_parent(tree, clade.root.name))
parent_rec = tree_df[tree_df.index.str.contains(parent_node.name)]
clade = parent_node
clade_df = clade_df.append(parent_rec)

# Prune clades outside
prune_nodes = []

for c in clade.find_clades(order="postorder"):
    if c.name not in clade_df.index:
        prune_nodes.append(c)
        
for node in prune_nodes:
    if node in clade.find_clades(node):
        try:
            clade.prune(node)
        except AttributeError as e:
            print(e)
            continue

# Clean up dataframe
for node in clade_df.index:
    if node not in [c.name for c in clade.find_clades()]:
        clade_df.drop(node, inplace=True)


display(clade_df)

## Convert Dataframe to Geopandas

In [ ]:
# Convert dataframe to geopandas
geometry = []
for rec in clade_df.iterrows():
    sample = rec[0]
    x = clade_df["Mugration_" + geo_attr + "_Lon"][sample] 
    y = clade_df["Mugration_" + geo_attr + "_Lat"][sample]
    geometry.append(shapely.geometry.Point(x,y))

clade_gdf = geopandas.GeoDataFrame(clade_df, crs=CRS, geometry=geometry)   

## Add Nodes

In [ ]:
# ---------------------------------------
# Root Data
root_geom = clade_gdf["geometry"][clade.root.name]
root_date = clade_gdf["timetree_num_date"][clade.root.name]
root_state = clade_df["Mugration_Branch_Major"][clade.root.name]
root_color = colors_dict["branch_major"][root_state] 
root_marker = lines.Line2D([root_geom.x], 
            [root_geom.y],
            marker="*",
            markerfacecolor=root_color, 
            markeredgecolor="black", 
            markeredgewidth=0.5,
            markersize=10, 
            zorder=4,
        )
# ---------------------------------------
# Node and Connections Data
network = tree2network(clade)
network_connections = []
node_markers = []

for connection in network:
    # Node names
    origin_name = connection[0].name
    dest_name = connection[1].name
    
    # Node Geometry
    origin_geom = clade_gdf["geometry"][origin_name]
    dest_geom = clade_gdf["geometry"][dest_name]
    
    # Aesthetic
    dest_state = clade_gdf["Mugration_Branch_Major"][dest_name]
    dest_color = colors_dict["branch_major"][dest_state] 
    dest_confidence = clade_gdf["Mugration_" + geo_attr + "_Confidence"][dest_name]
    dest_geo   = clade_df["Mugration_" + geo_attr][dest_name]
    dest_size  = sizes_dict[geo_attr.lower()][dest_geo]     
            
    if dest_confidence < MUG_CONF_THRESH:
        dest_color = "white"
    
    # Create the connection line
    if origin_geom != dest_geom:        
        connection = lines.Line2D(
           [origin_geom.x, dest_geom.x], 
           [origin_geom.y, dest_geom.y],  
           color=dest_color,
           lw=1.5,
           #transform=ccrs.Geodetic(),
           path_effects=[
               patheffects.Stroke(linewidth=2, foreground='black'),
               patheffects.Normal(),
           ],
           zorder=2,
           label="{},{}".format(origin_name, dest_name),
        )        
        network_connections.append(connection) 
        #network_connect_uniq.append([origin_geom, dest_geom])

## Animate

In [ ]:
min_date = math.floor(min(clade_df["timetree_num_date"]))
min_date = int(math.floor(min_date / 100.0)) * 100

max_date = math.ceil(max(clade_df["timetree_num_date"]))
max_date = int(math.ceil(max_date / 100.0)) * 100
tick_step = 100
frame_step = 5
time_xticks = range(min_date, max_date + 1, tick_step)
animate_frames = range(min_date, max_date + 1, frame_step)
# Keys are frames

animate_dict = {}
for i in range(0,len(animate_frames)):
    animate_dict[i] = animate_frames[i]

total_frames = len(animate_dict)
fps = int(100 / frame_step) # frames per second
speed = 1
spf = 1000 / (fps * speed) # (milli)seconds per frame

print(min_date, max_date, tick_step)
#print(animate_dict)

In [ ]:
%matplotlib notebook

fig_maptime = plt.figure()
gs          = gridspec.GridSpec(2, 1, figure=fig_maptime, 
                                wspace=0, 
                                hspace=-0.5, 
                                height_ratios=[20, 1])
ax_map      = fig_maptime.add_subplot(gs[0, 0], projection=ccrs.PlateCarree())
ax_time     = fig_maptime.add_subplot(gs[1, 0])

# Customize Map Axis
#ax_map.add_image(BASEMAP, 4)
world_polygons.plot(ax=ax_map, color="grey", alpha=0.5, ec="black", lw=0.5)
mb = Polygon(clade_gdf['geometry']).bounds
xb, yb = 10, 8
map_bounds = [mb[0] - xb, mb[2] + xb, mb[1] - yb, mb[3] + yb]
ax_map.set_extent(map_bounds, crs=ccrs.PlateCarree()) 

# Customize Time Axis
ax_time.set_xticks(time_xticks)
ax_time.set_xlim(min_date, max_date)
ax_time.set_ylim(0,0.1)
ax_time.set_yticks([])
[ax_time.axvline(x, color="black", alpha=1) for x in time_xticks]

time_line = ax_time.axvline(x=0, color='grey', lw=5, alpha=0.75)

def animate_init():
    time_line.set_data([],[])
    return time_line,

def animate_update(frame, progressive=False):
    frame_date = animate_dict[frame]

    # Timeline animation
    time_line.set_data([frame_date,frame_date], [0,1])
            
    # Check for connections
    for connection in network_connections: 
        origin_name = connection.get_label().split(",")[0]
        origin_date = clade_df["timetree_num_date"][origin_name]
        dest_name = connection.get_label().split(",")[1]
        dest_date = clade_df["timetree_num_date"][dest_name]
        dest_geom = clade_gdf["geometry"][dest_name]
        dest_geo   = clade_df["Mugration_" + geo_attr][dest_name]
        dest_size  = sizes_dict[geo_attr.lower()][dest_geo] 
        
        # Plot the root
        if frame == 0:
            ax_map.add_line(copy.copy(root_marker))
        # This marker will be plotted in future animation
        if origin_date > frame_date: continue
       
        num_seg = math.ceil((dest_date - origin_date) / frame_step) 
        date_seg_delta = (dest_date - origin_date)/ num_seg
        
        marker = lines.Line2D([dest_geom.x], 
                [dest_geom.y],
                marker="o",
                markerfacecolor=dest_color, 
                markeredgecolor="black", 
                markeredgewidth=0.5,
                markersize=dest_size, 
                zorder=3,
                label=dest_name,                              
            )
                
        # Option 1) Non-Progressive Lines
        if not progressive or num_seg <= 1:
            if dest_date <= frame_date:
                # Check if it was previously plotted
                if dest_date > animate_dict[frame - 1]:
                    ax_map.add_line(copy.copy(connection)) 
                    
        # Option 2) Progressive Lines
        else:
            conn_seg = segment_line(connection, num_seg, cumulative=True)
            for i in range(1,len(conn_seg) + 1):
                seg_date = origin_date + (date_seg_delta * i)
                if seg_date <= frame_date:
                    ax_map.add_line(copy.copy(conn_seg[i - 1]))
                        
        # Add Markers
        if dest_date > frame_date: continue
        # Check if it was previously plotted
        if dest_date <= animate_dict[frame - 1]: continue
        ax_map.add_line(copy.copy(marker))                
                                                 
    return time_line,

#animate_init()
#for i in range(0,1):
#     animate_update(i, progressive=True)

print("Animation will be {} frames.".format(len(animate_dict)))
ani = animation.FuncAnimation(
    fig=fig_maptime, 
    func=animate_update,
    fargs=(True,),
    init_func=animate_init,
    interval=spf,
    #frames=len(animate_dict), 
    frames=20,
    blit=True,
    repeat=True
    )

# ani.save('maptime.gif', writer="imagemagick")
#plt.show()

In [ ]:
import sys, importlib
importlib.reload(sys.modules['config'])
from config import *

---
# Export

## Dataframe

In [ ]:
out_path_df = os.path.join(outdir, SCRIPT_NAME + ".tsv" )
out_path_pickle_df = os.path.join(outdir,  SCRIPT_NAME + ".df.obj" )

tree_df.to_csv(out_path_df, sep="\t")
with open(out_path_pickle_df,"wb") as outfile:
    dill.dump(tree_df, outfile)

## Timetrees

In [ ]:
# Phyloxml
tt_copy = copy.deepcopy(tt)
out_path_xml = os.path.join(outdir,  SCRIPT_NAME + "_timetree.xml" )
Phylo.write(tt_copy.tree, out_path_xml, 'phyloxml')

# Nexus
out_path_nexus = os.path.join(outdir, SCRIPT_NAME + "_timetree.nexus" )
Phylo.write(tt_copy.tree, out_path_nexus, 'nexus', format_branch_length='%1.{}f'.format(BRANCH_LEN_SIG_DIG))

# Dill object
out_path_dill_tree = os.path.join(outdir,  SCRIPT_NAME + "_timetree.treetime.obj" )
with open(out_path_dill_tree,"wb") as outfile:
    dill.dump(tt_copy, outfile)
    
# Newick (remove comments)
for c in tt_copy.tree.find_clades(): c.comment = None
out_path_nwk = os.path.join(outdir, SCRIPT_NAME + "_timetree.nwk" )
Phylo.write(tt_copy.tree, out_path_nwk, 'newick', format_branch_length='%1.{}f'.format(BRANCH_LEN_SIG_DIG))

## Divtrees

In [ ]:
tt_copy = copy.deepcopy(tt)
# Convert to divtree
for n in tt_copy.tree.find_clades():
    n.branch_length=n.mutation_length

# Phyloxml
out_path_xml = os.path.join(outdir,  SCRIPT_NAME + "_divtree.xml" )
Phylo.write(tt_copy.tree, out_path_xml, 'phyloxml')

# Nexus
out_path_nexus = os.path.join(outdir, SCRIPT_NAME + "_divtree.nexus" )
Phylo.write(tt_copy.tree, out_path_nexus, 'nexus', format_branch_length='%1.{}f'.format(BRANCH_LEN_SIG_DIG))

# Dill object
out_path_dill_tree = os.path.join(outdir,  SCRIPT_NAME + "_divtree.treetime.obj" )
with open(out_path_dill_tree,"wb") as outfile:
    dill.dump(tt_copy, outfile)
    
# Newick (remove comments)
for c in tt_copy.tree.find_clades(): c.comment = None
out_path_nwk = os.path.join(outdir, SCRIPT_NAME + "_divtree.nwk" )
Phylo.write(tt_copy.tree, out_path_nwk, 'newick', format_branch_length='%1.{}f'.format(BRANCH_LEN_SIG_DIG))

## Augur JSON

In [ ]:
augur_dict = augur_export(
    tree_path=out_path_nwk, 
    aln_path=aln_path, 
    tree=tt_copy.tree, 
    tree_df=tree_df, 
    color_keyword_exclude=["color", "coord", "lat", "lon"],
    type_convert = {
        "Branch_Number" : (lambda x : str(x))
    },
)

print(augur_dict["nodes"]["NODE0"])

out_path_augur_json = os.path.join(outdir, SCRIPT_NAME + "_augur.json" )
utils.write_json(data=augur_dict, file_name=out_path_augur_json, indent=JSON_INDENT)

## Auspice JSON

In [ ]:
auspice_dict = auspice_export(
    tree=tt_copy.tree, 
    augur_json_paths=[out_path_augur_json], 
    auspice_config_path=auspice_config_path, 
    auspice_colors_path=auspice_colors_path,
    auspice_latlons_path=auspice_latlon_path, 
    )

# Write outputs - For Local Rendering
out_path_auspice_local_json = os.path.join(outdir, SCRIPT_NAME + "_auspice.json" )
utils.write_json(data=auspice_dict, file_name=out_path_auspice_local_json, indent=JSON_INDENT, include_version=False)
export_v2.validate_data_json(out_path_auspice_local_json)
print("Validation successful for local JSON.")

# Write outputs - For Remote Rendering
out_path_auspice_remote_json = os.path.join(auspice_remote_dir_path, AUSPICE_PREFIX + SCRIPT_NAME.replace("_","-") + ".json" )
utils.write_json(data=auspice_dict, file_name=out_path_auspice_remote_json, indent=JSON_INDENT, include_version=False)
export_v2.validate_data_json(out_path_auspice_remote_json)
print("Validation successful for remote JSON.")